In [1]:
# pip install pymysql

In [2]:
# pip install ipython-sql

In [3]:
# pip install mysqlclient

In [4]:
# Importing libraries
import json
import pymysql
import os
import re
import pandas as pd
import mysql.connector as mysql
from collections import defaultdict
from datetime import datetime
import re
from pandas.io.json import json_normalize
import requests

In [5]:
# !kaggle datasets list -s 'FIFA World Cup'

In [6]:
# !kaggle datasets download -d 'abecklas/fifa-world-cup' --force

In [7]:
# !chmod 600 /Users/stephieeh/.kaggle/kaggle.json
# !unzip fifa-world-cup.zip -d Fifa_data/

In [8]:
# WorldCupMatches = pd.read_csv('Fifa_data/WorldCupMatches.csv')
# WorldCupPlayers = pd.read_csv('Fifa_data/WorldCupPlayers.csv')
# WorldCups = pd.read_csv('Fifa_data/WorldCups.csv')

In [9]:
# Connecting to database
conn =pymysql.connect(host="localhost",user="root",password="Jetaime1818",database="fifa")
cursor = conn.cursor()

In [10]:
# Extract data from database
WorldCupMatches=pd.read_sql_query("SELECT * FROM worldcupmatches", conn)
WorldCupPlayers=pd.read_sql_query("SELECT * FROM worldcupplayers", conn)
WorldCups=pd.read_sql_query("SELECT * FROM worldcups", conn)

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/1754521753.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  WorldCupMatches=pd.read_sql_query("SELECT * FROM worldcupmatches", conn)
/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/1754521753.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  WorldCupPlayers=pd.read_sql_query("SELECT * FROM worldcupplayers", conn)
/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/1754521753.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
 

Quick descriptive analysis to explore our data

In [11]:
# Distribution of cup matches per year
matchesPlayed=pd.read_sql_query("SELECT Year, COUNT(*) `Matches Played Count`\
FROM WorldCupMatches GROUP BY Year ORDER BY Year desc LIMIT 11", conn)
matchesPlayed

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/3902191190.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  matchesPlayed=pd.read_sql_query("SELECT Year, COUNT(*) `Matches Played Count`\


,Year,Matches Played Count
0,2014,80
1,2010,64
2,2006,64
3,2002,64
4,1998,64
5,1994,52
6,1990,52
7,1986,52
8,1982,52
9,1978,38


In [12]:
# Get the minimum and maximum value of the goals scored throughout the years
scoredGoals=pd.read_sql_query("SELECT MIN(GoalsScored) AS `Minimum Number of Goals`,\
MAX(GoalsScored) AS `Maximum Number of Goals` FROM WorldCups", conn)
scoredGoals

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/845853711.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scoredGoals=pd.read_sql_query("SELECT MIN(GoalsScored) AS `Minimum Number of Goals`,\


,Minimum Number of Goals,Maximum Number of Goals
0,70,171


We have three tables in this dataset, two of which we can connect together with IDs. WorldCupMatches and WorldCupPlayers have two related columns between them - MatchID and RoundID. Using JOIN, we can uniquely identify and combine the rows between the tables to manipulate data and draw insights.

The below shows quick analysis to illustrates the difference between Home and Away Team names before we dig further into our dataset.

In [13]:
# Home Team
homeTeam=pd.read_sql_query("SELECT a.MatchID, a.ROUNDID,\
`Home Team Name`, `Home Team Initials`, `Coach Name` \
FROM worldcupmatches a JOIN WorldCupPlayers b \
on a.MatchID=b.MatchID and a.ROUNDID=b.RoundID \
GROUP BY a.MatchID ORDER BY a.RoundID ", conn)
homeTeam

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/953912786.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  homeTeam=pd.read_sql_query("SELECT a.MatchID, a.ROUNDID,\


,MatchID,ROUNDID,Home Team Name,Home Team Initials,Coach Name
0,1096,201,France,FRA,LUQUE Juan (MEX)
1,1090,201,USA,USA,GOETINCK Hector (BEL)
2,1093,201,Yugoslavia,YUG,DE CARVALHO Pindaro (BRA)
3,1098,201,Romania,ROU,BRU Francisco (ESP)
4,1085,201,Argentina,ARG,OLAZAR Francisco (ARG)
...,...,...,...,...,...
103,1158,429,Sweden,SWE,NAGY Joszef (HUN)
104,1175,429,Hungary,HUN,RAPPAN Karl (AUT)
105,1152,429,Brazil,BRA,MEISSNER Vaclav (TCH)
106,1233,462,Germany FR,FRG,NAUSCH Walter (AUT)


In [14]:
# Away Team
awayTeam=pd.read_sql_query("SELECT a.MatchID, a.ROUNDID,\
`Away Team Name`, `Away Team Initials`, `Coach Name` \
FROM worldcupmatches a JOIN WorldCupPlayers b \
on a.MatchID=b.MatchID and a.ROUNDID=b.RoundID \
GROUP BY a.MatchID ORDER BY a.RoundID", conn)
awayTeam

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/792146094.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awayTeam=pd.read_sql_query("SELECT a.MatchID, a.ROUNDID,\


,MatchID,ROUNDID,Away Team Name,Away Team Initials,Coach Name
0,1096,201,Mexico,MEX,LUQUE Juan (MEX)
1,1090,201,Belgium,BEL,GOETINCK Hector (BEL)
2,1093,201,Brazil,BRA,DE CARVALHO Pindaro (BRA)
3,1098,201,Peru,PER,BRU Francisco (ESP)
4,1085,201,France,FRA,OLAZAR Francisco (ARG)
...,...,...,...,...,...
103,1158,429,Cuba,CUB,NAGY Joszef (HUN)
104,1175,429,Switzerland,SUI,RAPPAN Karl (AUT)
105,1152,429,Czechoslovakia,TCH,MEISSNER Vaclav (TCH)
106,1233,462,Austria,AUT,NAUSCH Walter (AUT)


The main objective is to integrate our tables further, perhaps a bit of cleansing and transformation to get as many informationn as possible into one table.

The dataset has a number of countries in Europe and outside, the focus here would be to concentrate on European countries to keep our tables clean and the overview of this analysis effective.

In [15]:
# European Team
EUTeam=pd.read_sql_query("SELECT c.Country, CONCAT(c.Country, ' ','Premier League') as `League Name`,\
a.Year, COUNT(DISTINCT a.`Home Team Name`) as `Team Numbers`,\
COUNT(DISTINCT a.Stage) as `Stage Numbers`,\
AVG(a.`Home Team Goals`) as `Average Home Team Goals`,\
AVG(a.`Away Team Goals`) as `Average Away Team Goals`,\
AVG(a.`Home Team Goals` + a.`Away Team Goals`) as `Average Goals`,\
AVG(a.`Home Team Goals` - a.`Away Team Goals`) as `Average Difference Goals`,\
SUM(a.`Home Team Goals` + a.`Away Team Goals`) as `Total Goals` FROM WorldCupMatches a JOIN WorldCupPlayers b on a.MATCHID=b.MATCHID and a.ROUNDID=b.ROUNDID JOIN WorldCups c WHERE c.Country in ('Sweden','Switzerland','Spain', 'Germany', 'Italy', 'England', 'France')\
GROUP BY c.Country,`League Name`, a.Year HAVING COUNT(DISTINCT a.Stage)>=5 ORDER BY c.Country,`League Name`, a.Year", conn)
EUTeam

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/2184359970.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  EUTeam=pd.read_sql_query("SELECT c.Country, CONCAT(c.Country, ' ','Premier League') as `League Name`,\


,Country,League Name,Year,Team Numbers,Stage Numbers,Average Home Team Goals,Average Away Team Goals,Average Goals,Average Difference Goals,Total Goals
0,England,England Premier League,1930,9,6,3.267831,0.633188,3.901019,2.634643,2680.0
1,England,England Premier League,1934,8,5,2.814815,1.292023,4.106838,1.522792,2883.0
2,England,England Premier League,1938,8,5,3.355643,1.279528,4.635171,2.076115,3532.0
3,England,England Premier League,1950,10,5,3.126362,0.849673,3.976035,2.276688,3650.0
4,England,England Premier League,1954,11,8,4.178947,1.196491,5.375439,2.982456,6128.0
5,France,France Premier League,1930,9,6,3.267831,0.633188,3.901019,2.634643,5360.0
6,France,France Premier League,1934,8,5,2.814815,1.292023,4.106838,1.522792,5766.0
7,France,France Premier League,1938,8,5,3.355643,1.279528,4.635171,2.076115,7064.0
8,France,France Premier League,1950,10,5,3.126362,0.849673,3.976035,2.276688,7300.0
9,France,France Premier League,1954,11,8,4.178947,1.196491,5.375439,2.982456,12256.0


The above table shows a combination of the three tables we have, with the first 2 joining by IDs and with the third table bringing other elements such as country to create a clear image of the descriptive analysis we are trying to achieve.

The aggregate functions are used here to perform basic calculation but yet effective. Starting with count distinctive to get the number of unique values for Home Teams and Stage, and finishing off with average and sum to calculate the goals between each league whilst ensuring we group by the same dimensions used in the select statement to summarise our data points. The having clause is used to filter the group rows of our data.

In [16]:
# Match results of all countries
matchResults=pd.read_sql_query("SELECT a.MatchID, CAST(a.Datetime AS CHAR(12)) AS Date,a.`Home Team Name`,\
a.`Away Team Name`, a.`Home Team Goals`, a.`Away Team Goals`,\
CASE WHEN `Home Team Goals` - `AWAY Team Goals`>0 THEN `Home Team Initials`\
WHEN `Home Team Goals` - `AWAY Team Goals`<0 THEN `Away Team Initials` else 'Tie' end as Winner,\
CONCAT(a.City, ' - ', a.Stadium) AS Description FROM worldcupmatches a JOIN WorldCupPlayers b on a.MatchID=b.MatchID and a.ROUNDID=b.RoundID \
GROUP BY a.MatchID, Date ORDER BY a.MatchID, Date", conn)
matchResults

/var/folders/zv/gfvqnk2x0972ygvfdf81g3z80000gn/T/ipykernel_71648/1585573538.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  matchResults=pd.read_sql_query("SELECT a.MatchID, CAST(a.Datetime AS CHAR(12)) AS Date,a.`Home Team Name`,\


,MatchID,Date,Home Team Name,Away Team Name,Home Team Goals,Away Team Goals,Winner,Description
0,1084,22 Jul 1930,Argentina,Chile,3,1,ARG,Montevideo - Estadio Centenario
1,1085,15 Jul 1930,Argentina,France,1,0,ARG,Montevideo - Parque Central
2,1086,19 Jul 1930,Argentina,Mexico,6,3,ARG,Montevideo - Estadio Centenario
3,1087,30 Jul 1930,Uruguay,Argentina,4,2,URU,Montevideo - Estadio Centenario
4,1088,26 Jul 1930,Argentina,USA,6,1,ARG,Montevideo - Estadio Centenario
...,...,...,...,...,...,...,...,...
103,1372,08 Jun 1958,Soviet Union,England,2,2,Tie,Gothenburg - Nya Ullevi
104,1386,08 Jun 1958,France,Paraguay,7,3,FRA,Norrk�Ping - Idrottsparken
105,1407,08 Jun 1958,Hungary,Wales,1,1,Tie,Sandviken - Jarnvallen
106,1415,08 Jun 1958,Sweden,Mexico,3,0,SWE,Solna - Rasunda Stadium


The above creates a set of conditions to return the winner of each match played as the final results.